# **Macroeconomia Aberta e DSGE: Fundamentos, Estimação e Aplicações**

João Ricardo Costa Filho \\
https://www.joaocostafilho.com

# **Polítca monetária em uma economia pequena aberta: análise de funções impulso-resposta**

Replicação do modelo com prêmio de risco do artigo [Monetary Policy and Exchange Rate Volatility in a Small Open Economy](https://academic.oup.com/restud/article-abstract/72/3/707/1553469).

\
Jordi Galí, Tommaso Monacelli, Monetary Policy and Exchange Rate Volatility in a Small Open Economy, The Review of Economic Studies, Volume 72, Issue 3, July 2005, Pages 707–734, https://doi.org/10.1111/j.1467-937X.2005.00349.x

\
Código adaptado de [Johannes Pfeifer](https://github.com/JohannesPfeifer/DSGE_mod/tree/master/Gali_Monacelli_2005).

## Instale na máquina "virtual" o Octave e o Dynare.

In [1]:
!sudo apt-get update &> /dev/null
!sudo apt-get install -y x13as &> /dev/null

In [2]:
!apt-get install octave &> /dev/null

In [3]:
!apt-get install dynare &> /dev/null

## Modelo

### Variáveis

In [4]:
%%writefile variaveis.mod

%----------------------------------------------------------------
% Variáveis endógenas
%----------------------------------------------------------------

var pih    ${\pi_h}$   (long_name='Inflação doméstica')
    x      $x$         (long_name='Hiato do produto')
    y      $y$         (long_name='PIB')
    ynat   ${\bar y}$  (long_name='PIB Natural')
    rnat   ${\bar r}$  (long_name='Taxa de juros natural')
    r      $r$         (long_name='Taxa de juros nominal')
    s      $s$         (long_name='Termos de troca')
    pi     ${\pi}$     (long_name='Inflação ao consumidor')
    p      $p$         (long_name='Índice de preços ao consumidor')
    ph     ${p_h}$     (long_name='Nível dos preços de bens domésticos')
    e      $e$         (long_name='Taxa de câmbio')
    ystar  ${y^*}$     (long_name='PIB mundial')
    pistar ${\pi^{*}}$ (long_name='Inflação mundial')
    n      ${n}$       (long_name='Horas trabalhadas')
    nx     ${nx}$      (long_name='Exportações líquidas')
    wp     ${w-p}$     (long_name='Salário real')
    a      $a$         (long_name='Produtividade doméstica')
    c      $c$         (long_name='Consumo doméstico')
;


%----------------------------------------------------------------
% Variáveis exógenas
%----------------------------------------------------------------

varexo eps_star ${\varepsilon^{*}}$ (long_name='Choque na produtividade global')
       eps_a    ${\varepsilon^{a}}$ (long_name='Choque na produtividade doméstica')
;

Writing variaveis.mod


### Parâmetros

In [5]:
%%writefile parametros.mod

%----------------------------------------------------------------
% Parâmetros
%----------------------------------------------------------------

parameters sigma $\sigma$      (long_name='Coeficiente de aversão relativa ao risco')
             eta $\eta$        (long_name='Substituição doméstico-importado')
           gamma $\gamma$      (long_name='Elasticidade de substituição entre os bens importados')
             phi $\varphi$     (long_name='Inverso da elasticidade-Frisch da oferta de trabalho')
         epsilon $\varepsilon$ (long_name='Elasticidade de substituição entre os bens domésticos')
           theta $\theta$      (long_name='Parâmetro do Calvo.')
            beta $\beta$       (long_name='Fator de desconto intertemporal')
           alpha $\alpha$      (long_name='grau de abertura/home bias')
          phi_pi $\phi_\pi$    (long_name='Sensibilidade da taxa de juros à variações na taxa de inflação')
            rhoa $\rho_a$      (long_name='Coeficiente de persistência da produtividade')
            rhoy $\rho_y$      (long_name='Coeficiente de persistência do PIB mundial')
;

%----------------------------------------------------------------
% Calibração
%----------------------------------------------------------------

sigma   = 1;
eta     = 1 ;
gamma   = 1;
phi     = 3;
epsilon = 6;
theta   = 0.75;
beta    = 0.99;
alpha   = 0.4;
phi_pi  = 1.5;
rhoa    = 0.9;
rhoy    = 0.86;

Writing parametros.mod


### Modelo

In [6]:
%%writefile modelo.mod

%--------------------------------------------------------------------------------------------------------------------------------------
% Modelo
%--------------------------------------------------------------------------------------------------------------------------------------

model(linear);

%%%% Definições auxiliares sobre parâmetros %%%%

# rho     = beta^(-1) - 1;                                               % rho é a taxa de desconto intermporal, definida após a equação 11 na página 712.
# omega   = sigma * gamma + ( 1 - alpha ) * ( sigma * eta - 1 );         % definido após a equação 27 na página 716.
# sigma_a = sigma / ( ( 1 - alpha ) + alpha * omega );                   % definido após a equação 29 na página 717.
# Theta   = ( sigma * gamma - 1 ) + ( 1 - alpha ) * ( sigma * eta - 1 ); % definido após a equação 30 na página 717.
# lambda  = ( 1 - ( beta * theta ) ) * ( 1 - theta ) / theta;            % definido após a equação 32 na página 717.
# Gamma   = ( 1 + phi ) / ( sigma_a + phi );                             % definido após a equação 35 na página 718.
# Psi     = -Theta * sigma_a / ( sigma_a + phi );                        % definido após a equação 35 na página 718.
# kappa_a = lambda * ( sigma_a + phi );                                  % definido após a equação 36 na página 718.

[name='Salário real - Eq (11)']
wp    = sigma * c + phi * n;

[name='Inflação ao consumidor - Eq (14)']
pi   = pih + alpha * ( s - s(-1) );

[name='Termos de troca - Eq 15']
s    = s(-1) + e - e(-1) + pistar - pih;


[name='Função de produção - Eq (22)']
y      = a + n;

[name='Consumo - Eq (27)']
y      = c + alpha * omega / sigma * s;

[name='Restrição internacional - Eq (29)']
y      = ystar + 1 / sigma_a * s;

[name='Exportações líquidas - Eq (31)']
nx     = alpha * ( omega / sigma - 1 ) * s;

[name='Hiato do produto']
x      = y - ynat;

[name='PIB Potencial - Eq (35)']
ynat   = Gamma * a + alpha * Psi * ystar;

[name='Oferta agregada / curva de Phillips - Eq (36)']
pih    = beta * pih(+1)+ kappa_a * x;

[name='Curva IS - Eq (37)']
x      = x(+1) - sigma_a^(-1)*(r - pih(+1) - rnat) ;

[name='Taxa de juros natural']
rnat   = -sigma_a * Gamma * ( 1 - rhoa ) * a + alpha * sigma_a * ( Theta + Psi ) * ( ystar(+1) - ystar );

[name='stochastic process for technology p. 723']
a      = rhoa*a(-1) + eps_a;

[name='stochastic process for foreign output p. 723']
ystar  = rhoy*ystar(-1) + eps_star;

[name='Constant world inflation, see p.724 (Given constant world prices)']
pistar = 0;

[name='Regra de política monetária']
r      = phi_pi * pi;

%%%% Definições sobre variáveis %%%%

[name='Taxa de inflação ao consumidor']
pi   = p - p(-1);

[name='Taxa de inflação de bens domésticos']
pih  = ph - ph(-1);

end;

Writing modelo.mod


## Equilíbrio e funções impulso-resposta

In [21]:
%%writefile equilibrio.mod

@#include "variaveis.mod"

@#include "parametros.mod"

@#include "modelo.mod"

%--------------------------------------------------------------------------------------------------------------------------------------
% Equilíbrio
%--------------------------------------------------------------------------------------------------------------------------------------

resid;
steady;
check;

%--------------------------------------------------------------------------------------------------------------------------------------
% Funções impulso-resposta
%--------------------------------------------------------------------------------------------------------------------------------------

shocks;

var eps_a;
stderr 1;
end;

stoch_simul(ar=5, irf=16, graph_format = pdf);

Overwriting equilibrio.mod


In [8]:
%%writefile solve.m
dynare equilibrio.mod

Writing solve.m


In [22]:
!octave solve.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 18 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 2).
Processing outputs ...
done
Preprocessing completed.





Residuals of the static equations:

Equation number 1 : 0 : Salário real - Eq (11)
Equation number 2 : 0 : Inflação ao consumidor - Eq (14)
Equation number 3 : 0 : Termos de troca - Eq 15
Equation number 4 : 0 : Função de produção - Eq (22)
Equation number 5 : 0 : Consumo - Eq (27)
Equation number 6 : 0 : Restrição internacional - Eq (29)
Equation number 7 : 0 : Exportações líquidas - Eq (31)
Equation number 8 : 0 : Hiato do produto
Equation number 9 : 0 : PIB Potencial - Eq (35)
Equation number 10 : 0 : Oferta agregada / curva de Phillips - Eq (36)
Equation number 11 : 0 : Curva IS - Eq (37)
Equat